In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
tar = pd.read_excel('tar.xlsx')
con = pd.read_excel('controls.xlsx')

In [3]:
tar['TDEBT_ast'] = tar['total_liabilities'] / tar['total_assets']
tar['LTDEBT_ast'] = tar['long_term_liabilities'] / tar['total_assets']
tar['CFIN_ast'] = tar['total_cash_st_investment'] / tar['total_assets']
tar['CNFIN_ast'] = (tar['total_cash_st_investment'] - tar['current_liabilities']) / tar['total_assets']

In [4]:
con['TDEBT_ast'] = con['total_liabilities'] / con['total_assets']
con['LTDEBT_ast'] = con['long_term_liabilities'] / con['total_assets']
con['CFIN_ast'] = con['total_cash_st_investment'] / con['total_assets']
con['CNFIN_ast'] = (con['total_cash_st_investment'] - con['current_liabilities']) / con['total_assets']

In [5]:
tar20 = tar[tar['percent'] >= 20]
con20 = con[con['stake_20'] == 'yes']

In [7]:
tar20.head()

,Company,id,stake,percent,market_capitalization,year,net_income,c&e,total_assets,total_liabilities,total_equity,current_assets,current_liabilities,long_term_debt,total_cash_st_investment,Industry Group,Industry Sector,Primary Industry Classification,long_term_liabilities,MTB,tobin_q,TDEBT,LTDEBT,CFIN,CNFIN,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,TDEBT_ast,LTDEBT_ast,CFIN_ast,CNFIN_ast
0,"Echo Global Logistics, Inc. (NasdaqGS:ECHO)",1.0,majority,100.0,834.0,2021,15.8,41.3,963.9,564.58,399.4,493.48,377.14,133.95,41.30,Transportation,Industrials,Air Freight and Logistics,187.44,2.088132,1.450960,1.413570,0.469304,0.103405,-0.840861,NaN,NaN,NaN,NaN,NaN,0.585725,0.194460,0.042847,-0.348418
1,I-ON Digital Corp. (OTCPK:IONI),2.0,majority,100.0,10.6,2021,1.6,4.5,13.2,3.86,9.3,11.56,3.86,-,5.29,Software and Services,Information Technology,Application Software,0.00,1.139785,1.095455,0.415054,0.000000,0.568817,0.153763,NaN,NaN,NaN,NaN,NaN,0.292424,0.000000,0.400758,0.108333
2,"Five9, Inc. (NasdaqGM:FIVN)",3.0,majority,100.0,11906.8,2021,-42.1,220.4,1063.7,784.58,279.2,689.12,103.56,643.32,603.54,Software and Services,Information Technology,Application Software,681.02,42.646132,11.931353,2.810100,2.439183,2.161676,1.790759,NaN,NaN,NaN,AVG,STD,0.737595,0.640237,0.567397,0.470039
3,"Medallia, Inc. (NYSE:MDLA)",4.0,majority,100.0,5289.0,2021,-148.7,428.3,1401.6,893.41,508.2,918.76,387.18,448.06,682.39,Software and Services,Information Technology,Application Software,506.23,10.407320,4.410966,1.757989,0.996124,1.342759,0.580893,NaN,NaN,MTB,-137.278,3187.81,0.637422,0.361180,0.486865,0.210624
4,"Cloudera, Inc.",5.0,majority,100.0,3780.6,2021,-162.7,298.7,2508.6,1380.67,1127.9,997.92,663.11,487.09,596.39,Software and Services,Information Technology,Application Software,717.56,3.351893,2.057430,1.224107,0.636191,0.528761,-0.059154,NaN,NaN,TDEBT,-4.38875,135.728,0.550375,0.286040,0.237738,-0.026597


In [8]:
tar20_df = tar20[['Company', 'id', 'tobin_q', 'TDEBT_ast', 'LTDEBT_ast', 'CFIN_ast', 'CNFIN_ast', 'total_assets']]
con20_df = con20[['Company', 'id', 'tobin_q', 'TDEBT_ast', 'LTDEBT_ast', 'CFIN_ast', 'CNFIN_ast', 'total_assets']]

In [9]:
tar20_df['label'] = 1 #target company with label 1
con20_df['label'] = 0 #non-target company with lable 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
data = pd.concat([tar20_df, con20_df])

In [11]:
data.reset_index(drop=True, inplace=True)

In [12]:
data.head()

,Company,id,tobin_q,TDEBT_ast,LTDEBT_ast,CFIN_ast,CNFIN_ast,total_assets,label
0,"Echo Global Logistics, Inc. (NasdaqGS:ECHO)",1.0,1.450960,0.585725,0.19446,0.0428468,-0.348418,963.9,1
1,I-ON Digital Corp. (OTCPK:IONI),2.0,1.095455,0.292424,0,0.400758,0.108333,13.2,1
2,"Five9, Inc. (NasdaqGM:FIVN)",3.0,11.931353,0.737595,0.640237,0.567397,0.470039,1063.7,1
3,"Medallia, Inc. (NYSE:MDLA)",4.0,4.410966,0.637422,0.36118,0.486865,0.210624,1401.6,1
4,"Cloudera, Inc.",5.0,2.057430,0.550375,0.28604,0.237738,-0.0265965,2508.6,1


In [22]:
X_1 = data[['tobin_q', 'TDEBT_ast', 'LTDEBT_ast', 'CFIN_ast', 'CNFIN_ast', 'total_assets']].astype('float')
X_1 = sm.add_constant(X_1)
y = data['label']
model_1 = sm.Logit(y, X_1).fit(maxiter=100000)

Optimization terminated successfully.
         Current function value: 0.680122
         Iterations 6


In [23]:
print(model_1.summary())

                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                  690
Model:                          Logit   Df Residuals:                      684
Method:                           MLE   Df Model:                            5
Date:                Thu, 02 Dec 2021   Pseudo R-squ.:                 0.01879
Time:                        15:40:20   Log-Likelihood:                -469.28
converged:                       True   LL-Null:                       -478.27
Covariance Type:            nonrobust   LLR p-value:                  0.002979
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.2760      0.110      2.512      0.012       0.061       0.491
tobin_q          0.0003      0.000      0.694      0.488      -0.000       0.001
TDEBT_ast        0.0063      0.006      1.05

In [38]:
np.diag(model.cov_params())

array([1.20702580e-02, 1.54713453e-07, 1.74052160e+13, 1.74052160e+13,
       1.74052160e+13, 1.74052160e+13, 3.83163709e-11])

In [40]:
X_2 = data[['tobin_q', 'TDEBT_mar', 'LTDEBT_mar', 'CFIN_mar', 'CNFIN_mar', 'total_assets']].astype('float')
X_2 = sm.add_constant(X_2)
model_2 = sm.Logit(y, X_2).fit(maxiter=100000)

Optimization terminated successfully.
         Current function value: 0.680757
         Iterations 15


In [ ]:
tar20